<a href="https://colab.research.google.com/github/vizcayal/Thesis_UT/blob/V1/BASE_CASE_V7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing packages and Importing libraries**

In [112]:
!pip install gekko
!pip install ipynb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import time
import math
import datetime
from random import sample
from multiprocessing import *
from gekko import GEKKO
import pandas as pd
import random 
import os
from google.colab import files
from google.colab import drive
drive.mount("/content/drive")
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')
start_time = time.time()
# Main Program
 
#Upload distances from each hexagon to other
Hardwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Saw_V7.csv")
Hardwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hardwood_Pulp_V7.csv")
Softwood_Saw_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Saw_V7.csv")
Softwood_Pulp_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Softwood_Pulp_V7.csv")
Dist_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Dist_Mill.csv")
Cand_Dist = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Candidate_Distance.csv")
Mills_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Mills3.csv")
Factor_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Hexagon_Ratio.csv")
Growth_Data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Growth_Hexagon_V3.csv")
Growth_Data = Growth_Data[(Growth_Data['OWNERSHIP']=='PRIVATE')&(Growth_Data['SLOPE']=='LE40')] #FILTER OWNERSHIP AND SLOPE
 
i = Mills_Data[Mills_Data.Unique_ID==86399].index #removing this mills with no data
Mills_Data.drop(i, inplace=True)

Dist_Data['DIST'] = Dist_Data['DIST'].replace(0,0.98)   #REPLACE VALUES '0' BY '0.98'



**SAWMILL CLASS**

In [115]:
DISTANCE_MODE = 1     #Harvesting mode that reach closest locations
DENSITY_MODE = 2      #Harvesting mode that reach more dense locations
MAX_RADIO = 100        #Maximum Radio distance from mills
RADIO_MILL = 100        #Radio distance from mills 
RADIO_REFINERY = 100  #Radio distance from mills 
RESIDUES_LEFT = 0.3   #Percentage residues left in the ground
TRANSP_COST = 0.054   #Transportation cost per unit
COMMERCIAL_PERC = 0.338
NIPS_PERC =0.64
FOLDER_NAME = '/content/drive/My Drive/Colab Notebooks/Results/'

STUMPAGE_COST = [34,17,32,16]     #$/Ton [Hardwod Sawtimber, Hardwood Pulpwood, Softwood Sawtimber, Softwood Pulpwood]
HARVESTING_COST = [368.1,363.5,152.1,158.8]              #$/Acre
HAULING_COST = 0.054               #$/Ton/mi


YEARS = 1
MAX_MILLS = 1
CASE = 2

if CASE == 1:
  WILLING_NIPS = 0.38
  HARVESTING_MODE = DISTANCE_MODE
  FILE_NAME = 'C1_'
elif CASE == 2:
  WILLING_NIPS = 0.45
  HARVESTING_MODE = DISTANCE_MODE
  FILE_NAME = 'C2_'
elif CASE == 3:
  WILLING_NIPS = 0.65
  HARVESTING_MODE = DISTANCE_MODE
  FILE_NAME = 'C3_'
elif CASE == 4:
  WILLING_NIPS = 0.38
  HARVESTING_MODE = DENSITY_MODE
  FILE_NAME = 'C4_'
elif CASE == 5:
  WILLING_NIPS = 0.45
  HARVESTING_MODE = DENSITY_MODE
  FILE_NAME = 'C5_'
elif CASE == 6:
  WILLING_NIPS = 0.65
  HARVESTING_MODE = DENSITY_MODE
  FILE_NAME = 'C6_'

elif CASE == 7:
  WILLING_NIPS = 0.84
  HARVESTING_MODE = DISTANCE_MODE
  FILE_NAME = 'C7_'
elif CASE == 8:
  WILLING_NIPS = 0.84
  HARVESTING_MODE = DENSITY_MODE
  FILE_NAME = 'C8_'

elif CASE == 9:
  WILLING_NIPS = 0.49
  HARVESTING_MODE = DISTANCE_MODE
  FILE_NAME = 'C9_'
elif CASE == 10:
  WILLING_NIPS = 0.49
  HARVESTING_MODE = DENSITY_MODE
  FILE_NAME = 'C10_'


WILLING_FACTOR = NIPS_PERC*WILLING_NIPS + COMMERCIAL_PERC #Willingness to harvest factor

class Inventory:
  Total_Harvested = []      #Total Wood harvested in the first stage. used to determine forest residues
  Total_Residues = []       #Forest Residues Generated in each year
  Total_Acum_Residues = []  #Residues acumulated through the studied period equals residues from last month plus residues in the present month minus residues harvested 
  Res_Harvested = []        #Residues Harvested by the refinery in each year in the second stage
  Total_Cost = []         #Harvesting costs in the first stage by year
  Total_Cost_Res = []     #Harvesting cost in the second stage by cand and year
  Cand_Cost = pd.DataFrame() 
  

  def __init__(self, Av, Dist, Typ,Wt=-1, Pt=-1,Name ="", Willingness = 1,Factors = []):
    self.AcumResidues = []  #Residues generated by harvesting
    self.Available = list()
    self.Available.append(Av.copy()) #Dataframe that contains Available wood Inventory at diferent time t
    self.Available[0].loc[:,('SumOfTotal')] = self.Available[0].loc[:,('SumOfTotal')] * Willingness
    self.Available_Mill = [] #Dataframe that contains the Available Inventory close the mill to be harvested
    self.By_Mill = [] #Quantity harvested by each mills in each hexagon
    self.By_Cand_Harv = [] #Quantity harvested by each mills in each hexagon
    self.By_Cand_Avail = [] #Quantity harvested by each mills in each hexagon
    self.Candidates=[] #Candidate location for biorefinery
    self.Costs_1st = []
    self.Costs_2nd = []
    self.Harvest_Cost=[] #annual cost for 1st stage
    self.Dist = Dist #Matrix Distance from any Mills or candidates cells to other cells
    self.Dist_Mill = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Dist_Refi = [] #Dataframe that contains the Distance to the mill to be harvested
    self.Harvested = [] #Ammount of harvested wood
    self.Harvested_Res = []  #Residues Harvested
    self.Name = Name
    self.ProdType = Pt #type of product generated (pulp or saw)
    self.RefCost = [] #Matrix Distance from any Mills and candidates cells to other cells
    self.Residues = []  #Residues generated by harvesting
    self.Residues_Left = 0.30 #portion of residues left in the ground and not used for the biorefinery
    self.Results_2nd = []
    self.Typ =Typ       #type of wood 1 = Hardwod Sawtimber, 2= Hardwood Pulpwood, 3 = Softwood Sawtimber, 4 = Softwood Pulpwood]
    self.Willingness = Willingness #portion of timber available for residues.
    self.WoodType = Wt #type of wood needed (softwood, hardwood)
    
    self.Available[0].SumOfTotal =self.Available[0].SumOfTotal * self.Willingness #applying willingness to harvest
    
    Id_li = list(self.Available[0].loc[:,("Unique_ID")])
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    Data = {'Unique_ID':Id_li}
    self.By_Mill =pd.DataFrame(Data).copy()
    
    Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
    self.Harvested.append(pd.DataFrame(Data).copy())
    Data = {'Unique_ID':Id_li,'Growth':Ha_li}
    self.Growth =pd.DataFrame(Data).copy()    
    self.Factors = self.Available[0].merge(Factors, on='Unique_ID',how = 'left')
    pd.set_option("display.max_rows", 2000, "display.max_columns", 5)
    

  def Area_Harvested(self, Harvested,Area = 5):
    Perc = (Harvested/self.Available[0].SumOfTotal)*Area
    Ans = np.sum(Perc)
    return(Ans)

  def Harvest_All_Mills(self,t,k,mode = DISTANCE_MODE):
  #Harvest k mills at year t for the first stage in year t
    self.Available.append(self.Available[t].copy())
    for i in range(k):
      print('Cosechando Sawmill:',i)
      self.Harvest_Mill(t,i,mode)
 
    self.Harvested.append(self.Harvested[t].copy()) 
    self.Harvested[t+1].loc[:,('Harvested')] = 0
    self.Residues.append(self.Harvested[t].loc[:,('Unique_ID','Harvested')].copy())
    Rati = self.Residues[t].merge(self.Factors, on='Unique_ID',how = 'left')
    self.Residues[t].loc[:,('Harvested')] = self.Residues[t].loc[:,('Harvested')]*Rati.Ratio #substract the portion of residues left in the ground
    self.Residues[t].loc[:,('Harvested')] = self.Residues[t].loc[:,('Harvested')]*self.Factors.loc[:,('Ratio')] #substract the portion of residues left in the ground
    self.Residues[t].loc[:,('Harvested')] = self.Residues[t].loc[:,('Harvested')]*(1-RESIDUES_LEFT) #substract the portion of residues left in the ground
    self.Residues[t].loc[:,('Harvested')] = round(self.Residues[t].loc[:,('Harvested')]*0.01814*0.55) #transforming CF to Dry Tons
    print('Residues:',np.sum(self.Residues[t].Harvested))


  def Add_Growth(self,t):
    #increase the available inventory of period t
    self.Available[t].loc[:,("SumOfTotal")] = self.Available[t].loc[:,("SumOfTotal")]+self.Growth['Growth']


  def Initialize_Totals(self,Y):
    self.__class__.Total_Harvested = []
    self.__class__.Total_Residues = []
    for i in range(Y+1):
      self.__class__.Total_Harvested.append(pd.DataFrame({'Unique_ID' : [],'Harvested':[]})) #initialize Total Harvested variable (for all the layers)
      self.__class__.Total_Residues.append(pd.DataFrame({'Unique_ID' : [],'Harvested':[]})) #Initialize Total Residues variable (for all the layers)
    self.__class__.Total_Cost = [0]*Y
    self.__class__.Total_Cost_Res = [0]*Y
    

  def Total_Harvested_Update(self,Y):
    for i in range(Y):
      self.Harvested[Y].loc[:,('Harvested')] = self.Harvested[Y].loc[:,('Harvested')] + self.Harvested[i].loc[:,('Harvested')]
    for i in range(Y+1):
      c = pd.concat([self.__class__.Total_Harvested[i], self.Harvested[i]]).groupby(['Unique_ID']).sum().reset_index() #updating Total Harvested
      self.__class__.Total_Harvested[i] = c.copy()


  def Total_Residues_Update(self,Y):
    for i in range(Y):
      self.Residues[Y].loc[:,('Harvested')] = self.Residues[Y].loc[:,('Harvested')] + self.Residues[i].loc[:,('Harvested')]
 
    for i in range(Y+1):
      c = pd.concat([self.__class__.Total_Residues[i], self.Residues[i]]).groupby(['Unique_ID']).sum().reset_index() #Acumulate Residues from other layers
      self.__class__.Total_Residues[i] = c.copy()
      #print('Available resources at:',i,np.sum(self.__class__.Total_Residues[i].Harvested))
 

  def Harvest_All_Mills_All_Years(self,Y,mode=DISTANCE_MODE):
    Max = MAX_MILLS
    Min = 1    
    N_Mills = len(self.Mills)
    if (N_Mills>Max):
      N_Mills = Max
    if (N_Mills<Min):
      N_Mills = Min
    
    self.Harvest_Cost=[0]*Y

    now = datetime.datetime.now()
    for y in range(Y):
      self.Costs_1st.append([0]*4)
      self.Harvest_All_Mills(y,N_Mills,mode)
      self.Add_Growth(y)
      self.By_Mill.to_csv(FOLDER_NAME+'DETAILS/'+FILE_NAME+self.Name+'_ByMills'+'_'+str(now.day)+'_'+' year'+str(y)+'.csv')  
    self.Residues.append(self.Residues[0].loc[:,('Unique_ID','Harvested')].copy())
    self.Residues[Y].loc[:,('Harvested')] = 0
    pd.DataFrame(self.Costs_1st).to_csv(FOLDER_NAME+FILE_NAME+self.Name + '_COST_YEAR.csv')  

 
  def Growth_Update(self,Data):
    N = len(self.Growth)
    if self.ProdType == 'Sawmill':
      cl = 1
    elif self.ProdType == 'Pulpmill':
      cl =2
    Data = Data[Data['CLASS'] == cl]
    for i in range(N):    
      Id = self.Growth.loc[i,("Unique_ID")]
      Row = Data[Data['Unique_ID'] == Id]
      if not Row.empty:
        if self.WoodType == 'H':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,4] + Row.iloc[0,5] + 0.33 * Row.iloc[0,8]
        if self.WoodType == 'S':
          self.Growth.loc[i,('Growth')] = Row.iloc[0,6] + Row.iloc[0,7] + 0.67 * Row.iloc[0,8]
    
 
  def Cell_Less(self, Id_Mill, t, Radio=  RADIO_MILL):
    #Given Time, Radio and a Mill ID this function returns a list of cells with Distance and Availability
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #Look for in Dist matrix cell with the ID and closer than Radio
    Ans.reset_index(inplace = True)
    Disp =[]
 
    for j in range(len(Ans)):
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Ans.loc[:,("AVAIL")] = Disp 
    return(Ans)
 
 
  def Cell_Less_Res(self, Id_Mill, t, Radio):
    #Given Time, Radio and a Mill ID this function returns a Dataframe of cells with Distance and Forest Residues available
    Ans = Cand_Dist[(Cand_Dist['ORIG'] == Id_Mill)&(Cand_Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    Ans.reset_index(inplace=True)  
    Disp =[]
 
    for j in range(len(Ans)):
      if self.__class__.Total_Acum_Residues[t][self.__class__.Total_Acum_Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.__class__.Total_Acum_Residues[t][self.__class__.Total_Acum_Residues[t]["Unique_ID"]==Ans.loc[j,('DEST')]].Harvested.copy()))
    Ans.loc[:,("AVAIL")] = Disp 
    return(Ans)
 
  
  def Mills_Update(self,M):
    i1 = (M['s_h'] == self.WoodType)&(M['plant'] == self.ProdType)
    self.Mills = M.loc[i1,:]
    Num = len(self.Mills)
    ind = [i for i in range(Num)]
    self.Mills.index = ind
 

  def Total_Available(self, Id_Mill,t, Radio=RADIO_MILL):
    #Given a Radio and a Mill ID this function returns a the total inventory Available
    Ans = self.Dist[(self.Dist['ORIG'] == Id_Mill)&(self.Dist['DIST'] <= Radio)] #CELLS EQUAL ID_MILL AND CLOSER THAN RADIO
    rows = Ans.shape[0]
    ind = [i for i in range(rows)]
    Ans.index=ind    
    Disp =[]
    for j in range(len(Ans)):
      if self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].empty:
        Disp.append(0)
      else:
        Disp.append(int(self.Available[t][self.Available[t]["Unique_ID"]==Ans.loc[j,('DEST')]].SumOfTotal.copy()))
    Total = np.sum(Disp)
    return(Total)
 
 
  def List_Available(self,t,L): #Given a list L of ID, the funtion returns a List of Availability
    Av = []
    for id in L:
      if self.Available[t][self.Available[t]["Unique_ID"]==id].empty:
        Av.append(0)
      else:
        a1 = int(self.Available[t][self.Available[t]["Unique_ID"]==id].SumOfTotal)  
        Av.append(a1)
    return(Av)


  def List_Available_Res(self,t,L): #Given a list L of ID, the funtion returns a List of Availability

    Av = []
    for id in L:
      if self.__class__.Total_Acum_Residues[t][self.__class__.Total_Acum_Residues[t]["Unique_ID"]==id].empty:
        Av.append(0)
      else:
        a1 = int(self.__class__.Total_Acum_Residues[t][self.__class__.Total_Acum_Residues[t]["Unique_ID"]==id].Harvested)  
        Av.append(a1)
    return(Av)


  def Residues_Produced(self,t,L): #Given a list L of ID, the funtion returns a List of Availability
    Av = []
    for id in L:
      if self.__class__.Total_Residues[t][self.__class__.Total_Residues[t]["Unique_ID"]==id].empty:
        Av.append(0)
      else:
        a1 = int(self.__class__.Total_Residues[t][self.__class__.Total_Residues[t]["Unique_ID"]==id].Harvested)  
        Av.append(a1)
    return(Av)


  def Load_Residues(self,Y): #Given the number of years the residues are loaded from files
    for y in range(Y+1):
      self.__class__.Total_Residues[y]= pd.read_csv(FOLDER_NAME+FILE_NAME+'RESIDUES'+str(y)+'.csv')


  def Evaluate_Potencial(self,Y): 
    Potential = [] #list of list to store year/mill/available/cost/
    Num = len(self.Candidates)
    
    for c in range(Num):
      print('Cand', c)
      Id1 =self.Candidates[c] #assign unique ID
      a =self.Cell_Less_Res(Id1,0,RADIO_REFINERY) #look for hexagons between radio distance
      Id_list = a.loc[:, ('DEST')]
      self.Dist_Refi = a.loc[:, ('DIST')]  #obtain column Distance from 'a'
        
      for y in range(Y): #harvesting each mill from year 1 to Y
        Res_Year =[0]*4
        Res_Year[0] = Id1 #storing Mill Id
        Res_Year[1] = y   #storing year  
        self.Available_Refi  = self.Residues_Produced(y,Id_list)
        Res_Year[2] = sum(self.Available_Refi)   #storing year
        cost = self.Available_Refi @ self.Dist_Refi
        Res_Year[3] = cost   #storing year
        Potential.append(Res_Year)
    pd.DataFrame(Potential).to_csv(FOLDER_NAME+FILE_NAME+'POTENCIAL_CAND.csv')  


  def Optimal_Location_Total2(self,Y, Part ,Demand = 10000): 
    #run Ncan locations from year 0 to year 'y'
    Id_li = list(self.__class__.Total_Residues[0].loc[:,("Unique_ID")])
    r1 = len(self.__class__.Total_Residues[0])
    Ha_li =[0]*r1
    Data = {'Unique_ID':Id_li,'Harvested':Ha_li}    
    self.Harvested_Res.append(pd.DataFrame(Data).copy())
    Data = {'Unique_ID':Id_li}   
    self.By_Cand_Harv =pd.DataFrame(Data).copy()
    self.By_Cand_Avail =pd.DataFrame(Data).copy()
    self.Harvested_Res.append(pd.DataFrame(Data).copy())
    Harv_List=[] #list of list to store harvested amount in each year
    Result_2nd = [] #list of list to store mill/available/cost/demand for each year
    
    Num = len(self.Candidates)
    L_Cand = range(Num)

    for y in range(Y):
      Harv_List.append(Ha_li)    
    self.RefCost = [0]*int(Num)
 
    for c in L_Cand:
      Ha_li =[0]*Y
      self.__class__.Total_Acum_Residues = []
      self.__class__.Total_Acum_Residues.append(self.__class__.Total_Residues[0].copy())
      self.__class__.Total_Acum_Residues[0].loc[:,('Harvested')] = 0
      self.__class__.Total_Cost_Res = [0]*Y
      print('Cand', c)
      Id1 =self.Candidates[c] #assign unique ID
      a =self.Cell_Less_Res(Id1,0,RADIO_REFINERY) #look for hexagons between radio distance
      self.Dist_Refi = a.loc[:, ('DEST','DIST')] 
      start_time_cand = time.time()
        
      for i in range(Y): #harvesting each mill from year 1 to Y
        self.__class__.Total_Acum_Residues.append(self.__class__.Total_Acum_Residues[i].copy())
        self.__class__.Total_Acum_Residues[i+1].loc[:,('Harvested')] =self.__class__.Total_Acum_Residues[i+1]['Harvested'] + self.__class__.Total_Residues[i]['Harvested'].copy()
        self.Harvested_Res[i].loc[:,('Harvested')] = 0
        self.Harvest_Refinery(i,c,Demand)
        self.__class__.Total_Acum_Residues[i+1].loc[:,("Harvested")] = self.__class__.Total_Acum_Residues[i+1].loc[:,("Harvested")] - self.Harvested_Res[i].loc[:,('Harvested')].copy()
        self.By_Cand_Harv[i] = self.Harvested_Res[i].loc[:,('Harvested')].copy()
        self.By_Cand_Avail[i] = self.__class__.Total_Acum_Residues[i+1].loc[:,('Harvested')].copy()
        self.Harvested_Res.append(self.Harvested_Res[i].copy())     
        self.Harvested_Res[i+1].loc[:,('Harvested')] = 0
        self.__class__.Cand_Cost.loc[:,(str(c))] =self.__class__.Total_Cost_Res.copy()
      print ("*********CAND ",str(c),' Took ', time.time() - start_time_cand, " TO RUN")
        
      pd.DataFrame(self.By_Cand_Harv).to_csv(FOLDER_NAME+'DETAILS/'+FILE_NAME+'CAND_'+str(Id1)+'RES_HARVESTED.csv')  
      pd.DataFrame(self.By_Cand_Avail).to_csv(FOLDER_NAME+'DETAILS/'+FILE_NAME+'CAND_'+str(Id1)+'RES_AVAILABLE.csv')  
    pd.DataFrame(self.Results_2nd).to_csv(FOLDER_NAME+FILE_NAME+'RESULTS_2ND.csv')  


  def Harvest_Refinery(self,t,Cand, Ref_Demand): #Evaluates Candidate location for refinery in year t
    m2 = GEKKO(remote=True)
    m2.options.solver = 3 #IPOPT solver
    m2.options.IMODE = 3 #steady state optimization
    Res_Year =[0]*5
    Id1 =self.Candidates[Cand] #assign unique ID
    Res_Year[0] = Id1 #storing Mill Id
    Res_Year[1] = t   #storing year
    Id_list = list(self.Dist_Refi.loc[:, ('DEST')])
    #print('Ids: ',Id_list)
    D = list(self.Dist_Refi.loc[:, ('DIST')])
    #print('Distancia:',D)
    self.Available_Refi  = self.List_Available_Res(t+1,Id_list)
    #print('Available',self.Available_Refi) 
    print('TOTAL Available',np.sum(self.Available_Refi)) 
    
    Num = len(self.Dist_Refi) #calculate the number of cells betwee 
  
    H = m2.Array(m2.Var,(Num),value = 0)
    Res_Year[2] = np.sum(self.Available_Refi) #storing availability
   
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Refi[i]  
      
    prod0 = m2.Intermediate(m2.sum([self.Dist_Refi.loc[i,('DIST')]*H[i] for i in range(Num)]))
    m2.Equation(Ref_Demand ==m2.sum([H[i] for i in range(0,len(H))]))
    m2.Obj(prod0) 
    m2.solve(disp=False,debug=False)
    flat = [item for l in H for item in l]
    Res_Year[3] = np.sum(flat) #storing availability
    a = self.Dist_Refi
    a['Harvested'] = flat
    ans = a
    
    
    self.Harvested_Res[t].loc[:,('Harvested')]=0
    
    for j in range(len(ans)):                                                                         
      if not(self.Harvested_Res[t][self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]].empty):
        a1 = self.Harvested_Res[t]["Unique_ID"]==ans.loc[j,('DEST')]
        a2 = int(np.where(a1)[0])
        self.Harvested_Res[t].loc[a2,("Harvested")] = ans.loc[j,('Harvested')]  
    

    res = np.sum(flat*self.Dist_Refi.loc[:,('DIST')])  #Delivery cost to refinery
    self.__class__.Total_Cost_Res[t] += res
    self.RefCost[Cand] += res
    print('Cost:', self.RefCost[Cand], ' Year: ',t)
    Res_Year[4] = res #storing cost
    self.Results_2nd.append(Res_Year)



  def Harvest_Mill(self,t, j,mode=DISTANCE_MODE):
    #Given a period t and a Mill j the function provide the harvested wood and substract it from available inventory
    m = GEKKO(remote=True)
    m.options.solver = 3                        #IPOPT solver
    m.options.IMODE = 3                         #steady state optimization
    Dem = self.Mills.loc[j,('Demand')]
    Id1 =self.Mills.loc[j,('Unique_ID')]        #look for mill in position j
    Name_Mill =self.Mills.loc[j,('Name')]       #look for mill in position j
    Rad = 10
    Ready = False
    r = 1
    if mode == DENSITY_MODE:  #In case of Density Criteria the dataframe is sort by availability
      b =self.Cell_Less(Id1,t+1,MAX_RADIO)
      b.sort_values(by='AVAIL',inplace = True,ascending=False)
    else:
      while((Rad<=MAX_RADIO) & (not Ready)):
        b =self.Cell_Less(Id1,t+1,Rad)
        Id_list = b.loc[:, ('DEST')]
        self.Available_Mill  = self.List_Available(t+1,Id_list)
        D1 = np.sum(self.Available_Mill)
        if D1 >= Dem:
          Ready = True
        else:
          r = len(b)
          Rad +=10
    
    a = b.iloc[:r,:]         #look for cells with radio less than rad.
    self.Dist_Mill = a.loc[:, ('DEST','DIST')]  #fill Distances to mills
    Id_list = a.loc[:, ('DEST')]
    c  = self.List_Available(t+1,Id_list)
    self.Available_Mill = c[:r]
    D1 = np.sum(self.Available_Mill)
    
    while (D1<=Dem): #expand the number of cells until inventory available is greater than demand
      r +=10
      a = b.iloc[:r,:]         #Expanding the number of cells to reach availability greater than demand.
      self.Dist_Mill = a.loc[:, ('DEST','DIST')]  #fill Distances to mills
      Id_list = a.loc[:, ('DEST')]
      c  = self.List_Available(t+1,Id_list)
      self.Available_Mill = c[:r]
      D1 = np.sum(self.Available_Mill)
    
    print('Inventory: ',self.Name,' Year: ',t, 'Mill: ',Id1,'**Demanda:', Dem, 'Disponible',D1,  "Radio",Rad )  
    Available_At_0  = self.List_Available(0,Id_list)
    
    Num = len(self.Dist_Mill)
    H = m.Array(m.Var,(Num),value = 0)
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    self.By_Mill[Name_Mill] = Ha_li
        
    for i in range(Num):
      H[i].lower = 0
      H[i].upper = self.Available_Mill[i]
      
    if (mode == DISTANCE_MODE):
      prod0 = m.Intermediate(m.sum([(HAULING_COST+TRANSP_COST* self.Dist_Mill.loc[i,('DIST')])*H[i] for i in range(Num)]))    
    elif (mode ==DENSITY_MODE):
      prod0 = m.Intermediate(-m.sum([self.Available_Mill[i]*H[i] for i in range(Num)]))
    else:
      print("ERROR: Harvesting Mode Incorrect..")
 
    m.Equation(Dem ==m.sum([H[i] for i in range(0,len(H))]))
    m.Obj(prod0) 
    m.solve(disp=False,debug=False)
    flat = [item for l in H for item in l]
    
    ans= self.Dist_Mill
    ans['Harvested'] = flat
    ans.reset_index(inplace = True)    
    ans.rename(columns={ 'DEST' : 'Unique_ID'},  inplace=True)
    ans.set_index('Unique_ID', inplace = True)
    Id_li = list(self.Available[0].loc[:,("Unique_ID")])
    r1 = len(self.Available[0])
    Ha_li =[0]*r1
    Data = {'Unique_ID':Id_li,'Harvested':Ha_li}
    Aux =pd.DataFrame(Data).copy()
    Aux.set_index('Unique_ID', inplace = True)
    Aux.update(ans)
    Aux.reset_index(inplace = True)
    self.Harvested[t].Harvested = self.Harvested[t].Harvested.add(Aux.Harvested,fill_value=0)
    self.Available[t+1].loc[:,("SumOfTotal")] = self.Available[t+1].loc[:,("SumOfTotal")] - Aux.loc[:,("Harvested")]
    

    Costs = [0,0,0,0]
    Costs[0] = Dem * STUMPAGE_COST[self.Typ]                 #STUMPAGE COSTS
    
    Har = np.sum(flat)
    Ava = np.sum(Available_At_0)
    Ar = np.divide(flat , Available_At_0)*5
    Ar = pd.Series(Ar).fillna(0).tolist()
    Ar= np.sum(Ar)
    
    print('Total Harvested: ', Har)
    print('Total Available: ', Ava)
    print('Area Harvested: ', Ar)
    Costs[1] = Ar * HARVESTING_COST[self.Typ]       #harvesting costs proportional to the harvested area
    
    Mill_Cost = np.sum(flat * self.Dist_Mill.DIST)
    Costs[2] = Mill_Cost * HAULING_COST            #Hauling costs

    Costs = np.nan_to_num(Costs)

    self.Costs_1st[t][0] = self.Costs_1st[t][0]+ Costs[0]                         #Accumulated Costs
    self.Costs_1st[t][1] = self.Costs_1st[t][1]+ Costs[1]
    self.Costs_1st[t][2] = self.Costs_1st[t][2]+ Costs[2]
    self.Costs_1st[t][3] = self.Costs_1st[t][3]+ Costs[0]+ Costs[1]+ Costs[2]

    print('self.Costs_1st ',self.Costs_1st)
    self.__class__.Total_Cost[t]  += Costs[0]+ Costs[1]+ Costs[2] 
    


**1ST PHASE - MILLS DEMAND - SHORTEST DISTANCE CRITERIA**


In [ ]:
print("Creating Inventories Objects and Initializing...")
Hardwood_Saw = Inventory(Hardwood_Saw_Data,Dist_Data,Wt='H',Pt ='Sawmill', Name = "Hardwood_Saw_v3", Willingness = WILLING_FACTOR,Factors =Factor_Data,Typ =0)
Hardwood_Pulp = Inventory(Hardwood_Pulp_Data,Dist_Data,Wt='H',Pt ='Pulpmill', Name = "Hardwood_Pulp_v3", Willingness = WILLING_FACTOR,Factors =Factor_Data, Typ =1)
Softwood_Saw = Inventory(Softwood_Saw_Data,Dist_Data,Wt='S',Pt ='Sawmill', Name = "Softwood_Saw_v3", Willingness = WILLING_FACTOR,Factors =Factor_Data, Typ = 2)
Softwood_Pulp = Inventory(Softwood_Pulp_Data,Dist_Data,Wt='S',Pt ='Pulpmill', Name = "Softwood_Pulp_v3", Willingness = WILLING_FACTOR,Factors =Factor_Data, Typ = 3)
Hardwood_Saw.Initialize_Totals(YEARS)

print("Updating Growth Rate...")

Hardwood_Saw.Growth_Update(Growth_Data)
Hardwood_Pulp.Growth_Update(Growth_Data)
Softwood_Saw.Growth_Update(Growth_Data)
Softwood_Pulp.Growth_Update(Growth_Data)
 
# Adjusting Mixed Mills, from one Mixed mill two mills are created. a hardwood, and a softwood acordding to the availability around the mill 
Mixed = Mills_Data[Mills_Data['s_h']=='S/H'].copy()
for i in range(len(Mixed)):
  Hw = 0
  Sw = 0
  
  Tt = 0
  Id1 = Mixed.iloc[i,1]
  Id2 = Mills_Data[Mills_Data['Unique_ID']==Id1].copy()
  if Mills_Data.iloc[i,2] == "Sawmill" :
    Hw = Hardwood_Saw.Total_Available(Id1,0,MAX_RADIO)
    Sw = Softwood_Saw.Total_Available(Id1,0,MAX_RADIO)
  elif Mills_Data.iloc[i,2] == "Pulpmill" :
    Hw = Hardwood_Pulp.Total_Available(Id1,0,MAX_RADIO)
    Sw = Softwood_Pulp.Total_Available(Id1,0,MAX_RADIO)
  else:
    Sw = Softwood_Pulp.Total_Available(Id1,0,MAX_RADIO)
  Tt = Hw + Sw
  Perc_Hw = Hw/Tt
  Id2['s_h'] ='S'
  Id2['Demand'] = Id2['Demand']*(1-Perc_Hw)
  Mills_Data.append(Id2)
  Mills_Data.loc[Mills_Data['Unique_ID']==Id1,('Demand')] = Mills_Data[Mills_Data['Unique_ID']==Id1].Demand*Perc_Hw
  Mills_Data.loc[Mills_Data['Unique_ID']==Id1,('s_h')] = 'H'
  
print('Updating Mills Data...')  
  
#updating Mills According the wood type and product type

np.sum(Mills_Data.Demand)
Hardwood_Saw.Mills_Update(Mills_Data)
Hardwood_Pulp.Mills_Update(Mills_Data)
Softwood_Saw.Mills_Update(Mills_Data)
Softwood_Pulp.Mills_Update(Mills_Data)
 

print('Harvesting...')  

Hardwood_Saw.Harvest_All_Mills_All_Years(YEARS,mode =HARVESTING_MODE)
Hardwood_Pulp.Harvest_All_Mills_All_Years(YEARS,mode =HARVESTING_MODE)
Softwood_Saw.Harvest_All_Mills_All_Years(YEARS,mode =HARVESTING_MODE)
Softwood_Pulp.Harvest_All_Mills_All_Years(YEARS,mode =HARVESTING_MODE)

#printing Harvesting cost (1st)
Data = {'Cost':Hardwood_Saw.__class__.Total_Cost}
pd.DataFrame(Data).to_csv(FOLDER_NAME+FILE_NAME+'Harvest_Cost.csv')



Hardwood_Saw.Total_Harvested_Update(YEARS)
Hardwood_Pulp.Total_Harvested_Update(YEARS)
Softwood_Saw.Total_Harvested_Update(YEARS)
Softwood_Pulp.Total_Harvested_Update(YEARS)
 
Hardwood_Saw.Total_Residues_Update(YEARS)
Hardwood_Pulp.Total_Residues_Update(YEARS)
Softwood_Saw.Total_Residues_Update(YEARS)
Softwood_Pulp.Total_Residues_Update(YEARS)
print('TOTAL RESIDUES', np.sum(Hardwood_Saw.__class__.Total_Residues[YEARS].Harvested))

for y in range(YEARS+1):
  print('TOTAL HARVESTED at Hardwood_saw at',y,'  ', np.sum(Hardwood_Saw.Harvested[y].loc[:,('Harvested')]))
for y in range(YEARS+1):
  print('TOTAL HARVESTED at Hardwood_Pulp at',y,'  ', np.sum(Hardwood_Pulp.Harvested[y].loc[:,('Harvested')]))
for y in range(YEARS+1):
  print('TOTAL HARVESTED at Softwood_saw at',y,'  ', np.sum(Softwood_Saw.Harvested[y].loc[:,('Harvested')]))
for y in range(YEARS+1):
  print('TOTAL HARVESTED at softwood_Pulp at',y,'  ', np.sum(Softwood_Pulp.Harvested[y].loc[:,('Harvested')]))
 
for y in range(YEARS+1):
  Hardwood_Saw.__class__.Total_Harvested[y].to_csv(FOLDER_NAME+FILE_NAME+'TOTAL_HARVESTED_'+str(y)+'.csv')
  Hardwood_Saw.__class__.Total_Residues[y].to_csv(FOLDER_NAME+FILE_NAME+'RESIDUES_'+str(y)+'.csv')
 
print ("*********1ST STAGE TOOK", time.time() - start_time, "TO RUN")

In [ ]:
start_time = time.time()
#print('2nd stage started at:', start_time)
ma = Cand_Dist['ORIG'].unique()

N_Cand = len(ma)
print('ncand[0]',ma[0])
Residues_Inv = Inventory(Hardwood_Saw_Data,Dist_Data,Wt='H',Pt ='Sawmill', Name = "Hardwood_Saw", Willingness = WILLING_FACTOR,Factors =Factor_Data)
Residues_Inv.Candidates = ma
Residues_Inv.Initialize_Totals(YEARS)
Residues_Inv.Load_Residues(YEARS)
Residues_Inv.Optimal_Location_Total2(YEARS,1, 400000)

print('Costs of Candidates:',Residues_Inv.RefCost)
Min_Pos =Residues_Inv.RefCost.index(min(Residues_Inv.RefCost))
print('Minimum costs at:',ma[Min_Pos])
print ("*********2ND STAGE TOOK", time.time() - start_time, "TO RUN") 
